In [ ]:
import mapsims

# Running on the fly simulations with mapsims

The `mapsims` package allows to generate maps on the fly, so they can be generated as needed inside other pipelines.

Once the Map-Based-Simulations group publishes a simulation release, it will provide configuration files which setup `mapsims` with all the correct parameters,
see for example the 2 configuration toml (like .ini files <https://github.com/toml-lang/toml>) files: 

In [ ]:
!cat common.toml

In [ ]:
!cat noise.toml

## Run mapsims

We load the configuration files and then we override whatever configuration option we need,
or we could create another TOML file and load it after `noise.toml`.
`num` is the realization number, i.e. it fixes the seed for all different channels,
the set of maps released were generated with `num=0`, we can set it to 1 to get a different set.

We always work 1 tube at a time and get 2 maps with a cross-correlated component given by the atmosphere:

In [ ]:
sim = mapsims.from_config(["common.toml", "noise.toml"],
                          override={"channels":"ST3", "output_folder":".", "num":1})

Finally call execute to generate and return the maps.

NSIDE is automatically set based on the channel, you can also override that by providing `nside` key to `override`.

In [ ]:
maps_1 = sim.execute()

In [ ]:
maps_1.keys()

In [ ]:
maps_1["ST3_LF1"].shape

To get another realization you can either create another `sim` object with a different `num`
or directly override it on a already existing object:

In [ ]:
sim.num = 2 # this is only used for the output filename
sim.other_components["noise"].seed = 2 # this is the actual realization number
maps_2 = sim.execute()

To change tube without creating another `sim` object, override `sim.channels`:

In [ ]:
sim.channels

In [ ]:
sim.channels = mapsims.so_utils.parse_channels("LT6")

In [ ]:
sim.execute().keys()

## Write output maps

In [ ]:
# save fits files instead
sim.execute(write_outputs=True)

In [ ]:
%ls *.fits

## Plot the output maps

In [ ]:
import healpy as hp

In [ ]:
%matplotlib inline

In [ ]:
hp.mollview(maps_1["ST3_LF1"][0], min=-50, max=50)

In [ ]:
hp.mollview(hp.ma(maps_1["ST3_LF1"][0]) - hp.ma(maps_2["ST3_LF1"][0]), min=-50, max=50)

## Generate multiple splits

Override `nsplits` to generate multiple splits, the output maps will be a 3 dimensional array where for each split we have the 3 IQU components.

In [ ]:
sim = mapsims.from_config(["common.toml", "noise.toml"],
                          override={"channels":"ST3", "output_folder":".", "num":1, "nsplits":4})

In [ ]:
maps = sim.execute()

In [ ]:
maps["ST3_LF1"].shape